In [2]:
import torch

# einsum
torch.einsum(equation, *operands)<br>
공식홈페이지 : Sums the product of the elements of the input operands along dimensions specified using a notation based on the Einstein summation convention.

numpy, tensorflow에도 동일한 연산이 있음

In [14]:
def check(equation, *matrix):
    print('Before operating')
    for mat in matrix:
        print(mat)
    einsum = torch.einsum(equation, *matrix)
    print('After operating')
    print(einsum)

In [6]:
# 1개의 tensor에 대한 sum
test = torch.rand(2,3)
check()

tensor([[0.2648, 0.0273, 0.6172],
        [0.8527, 0.5430, 0.2363]])
tensor(2.5414)
